# Generate random networks

In [29]:
import os,random,glob

In [6]:
def randomG(n,k):
    arcs = [(src,dst) for src in range(n) for dst in range(src+1,n)]
    return random.sample(arcs,k)

In [19]:
randomG(5,7)

[(1, 4), (1, 2), (0, 4), (0, 3), (2, 3), (0, 2), (2, 4)]

## Now generate networks and data formats

In [30]:
netdn = 'artnets'
os.makedirs(netdn,exist_ok=True)

n=5
for g in range(100):
    for k in (4,7):
        arcs = randomG(n,k)
        bnfn = os.path.join(netdn,'k%d_%d.bn'%(k,g))
        bnf = open(bnfn,'w')
        bnf.write(str(n)+'\n')
        for a in sorted(arcs):
            bnf.write('%d %d\n' % a)
        bnf.close()

In [31]:
datdn = 'artdats'
os.makedirs(datdn,exist_ok=True)

n=5
for g in range(100):
    vdfn = os.path.join(datdn,'data_%d.vd'%g)
    vdf = open(vdfn,'w')
    for i in range(n):
        r = random.choice([2,3,4])
        vdf.write('V%d\t'%i + '\t'.join(map(str,range(r))) + '\n')
    vdf.close()

## And then generate parameters

In [34]:
from numpy.random import dirichlet
import numpy as np

In [33]:
dirichlet([0.5,0.5,0.5],4)

array([[ 0.35735531,  0.16236268,  0.480282  ],
       [ 0.42560868,  0.19558108,  0.37881025],
       [ 0.00525068,  0.20195296,  0.79279636],
       [ 0.42933045,  0.55704185,  0.0136277 ]])

In [56]:
def bnfn2parents(bnfn):
    bnf = open(bnfn)
    n = int(bnf.readline())
    parents = [set() for i in range(n)]
    for src,dst in (map(int, l.split()) for l in bnf):
        parents[dst].add(src)
    return parents

def parents2pccr(parents,vdfn):
    parents = bnfn2parents(bnfn)
    rs = [l.count('\t') for l in open(vdfn)]
    for i,r in enumerate(rs):
        q = np.product([rs[p] for p in parents[i]],dtype=int)
        yield q,r

In [57]:
parents = bnfn2parents('artnets/k4_1.bn')
print(parents)
print(list(parents2pccr(parents,'artdats/data_1.vd')))

[set(), {0}, set(), {0}, {1, 2}]
[(1, 2), (2, 4), (2, 2), (16, 2), (4, 4)]


In [91]:
for bnfn in glob.glob(os.path.join(netdn,'*.bn')):
    g = os.path.basename(bnfn)[3:-3]
    vdfn=os.path.join(datdn,'data_%s.vd'%g)
    parents = bnfn2parents(bnfn)

    for paramtype in ('Jeff','BDeu'):
        bnmf = open(bnfn[:-3]+'_%s.bnm'%paramtype, 'w')    
        for i,(q,r) in enumerate(parents2pccr(parents,vdfn)):
            bnmf.write('#@DENSE@\n')
            a = 0.5 if paramtype == 'Jeff' else max(1.0/r/q,0.01)
            bnmf.write('\t'.join(['V%d'%p for p in [i]+list(parents[i])])+'\n')
            #print([a]*r, q)
            params = dirichlet([a]*r,q)
            for row in params:
                bnmf.write('\t'.join(map(str,row))+'\n')
            bnmf.write('\n')
        bnmf.close()
    

        